In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

!whoami
!date

alibow
Thu Jun 18 14:27:13 PDT 2020


# Check model 6 NTD BP

In [2]:
output_dirs = ['/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/v7.0_prelim_nigeria/count_data/']

locations = ['Nigeria']

In [3]:
# available data frames

# births
# births_with_ntd
# birth_weight
# deaths
# gestational_age
# person_time
# population
# state_person_time
# transition_count
# ylds
# ylls

In [4]:
births = pd.read_hdf(output_dirs[0] + 'births.hdf')
births.head()

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births,357,baseline,76.0
1,2020,female,covered,live_births,357,baseline,0.0
2,2020,female,covered,live_births,357,baseline,11.0
3,2020,female,covered,live_births,357,folic_acid_fortification_scale_up,76.0
4,2020,female,covered,live_births,357,folic_acid_fortification_scale_up,0.0


In [6]:
births_by_scenario = births.groupby(['input_draw','scenario','sex']).sum().groupby(['scenario','sex']).mean()
births_by_scenario.head()

value
scenario                          sex            
baseline                          female  11073.5
                                  male    11416.0
folic_acid_fortification_scale_up female  11073.5
                                  male    11416.0
iron_fortification_scale_up       female  11073.5

In [7]:
ntd_births = pd.read_hdf(output_dirs[0] + 'births_with_ntd.hdf')
ntd_births.head()

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births_with_ntds,357,baseline,0.0
1,2020,female,covered,live_births_with_ntds,357,baseline,0.0
2,2020,female,covered,live_births_with_ntds,357,baseline,0.0
3,2020,female,covered,live_births_with_ntds,357,folic_acid_fortification_scale_up,0.0
4,2020,female,covered,live_births_with_ntds,357,folic_acid_fortification_scale_up,0.0


In [8]:
ntd_births_by_scenario = ntd_births.groupby(['input_draw','scenario','sex']).sum().groupby(['scenario','sex']).mean()
ntd_births_by_scenario

value
scenario                          sex          
baseline                          female   20.5
                                  male     18.0
folic_acid_fortification_scale_up female   20.5
                                  male     18.0
iron_fortification_scale_up       female   20.5
                                  male     18.0
vitamin_a_fortification_scale_up  female   20.5
                                  male     18.0

In [9]:
ntd_bp = ntd_births_by_scenario / births_by_scenario
ntd_bp

value
scenario                          sex             
baseline                          female  0.001851
                                  male    0.001577
folic_acid_fortification_scale_up female  0.001851
                                  male    0.001577
iron_fortification_scale_up       female  0.001851
                                  male    0.001577
vitamin_a_fortification_scale_up  female  0.001851
                                  male    0.001577

# Check artifact NTD BP

In [11]:
art = Artifact('/share/costeffectiveness/artifacts/vivarium_conic_lsff/nigeria.hdf', 
                   filter_terms=['year_start == 2017', 'age_start < 5'])
data = art.load('cause.neural_tube_defects.birth_prevalence')
data_mean = pd.DataFrame(data.mean(axis=1)).rename(columns={0:'value'})
data_limits = pd.DataFrame(data.quantile([0.025,0.975], axis=1)).transpose().rename(columns={0.025:'lower',
                                                                                             0.975:'upper'})
data = data_mean.merge(data_limits, right_index=True, left_index=True).reset_index()
data.head()

,location,sex,year_start,year_end,value,lower,upper
0,Nigeria,Female,2017,2018,0.002515,0.002195,0.002877
1,Nigeria,Male,2017,2018,0.002704,0.002321,0.003116


# Conclusion:

- Model 4 NTD BP was within GBD range, Model 6 NTD BP was below GBD range
- Model 4 had same number of live births for each scenario, model 6 did not